### Some useful and practical python skills/topics

`bokeh` example: https://docs.bokeh.org/en/latest/docs/gallery/texas.html

In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd

<a id='bokeh'></a>
### Interactive mapping


In [2]:
#We use an awesome package called bokeh (I love this package!)
from bokeh.io import output_file, show,output_notebook
from bokeh.models import ColumnDataSource,ColorBar,HoverTool
from bokeh.transform import linear_cmap
from bokeh.plotting import figure
from bokeh.palettes import Spectral6 #https://docs.bokeh.org/en/latest/docs/reference/palettes.html

In [3]:
IL_acs = gpd.read_file("IL_acs.shp")

DriverError: IL_acs.shp: No such file or directory

In [4]:
#To make your map be outputted inline
output_notebook()

Loading BokehJS ...

In [5]:
#Don't change this!!!
#Just copy this whole cell.
#This is a helper function for converting a GeoDataFrame to the format that bokeh can recognize.

def gpd_bokeh(df):
    """Convert geometries from geopandas to bokeh format"""
    nan = float('nan')
    lons = []
    lats = []
    for i,shape in enumerate(df.geometry.values):
        if shape.geom_type == 'MultiPolygon':
            gx = []
            gy = []
            ng = len(shape.geoms) - 1
            for j,member in enumerate(shape.geoms):
                xy = np.array(list(member.exterior.coords))
                xs = xy[:,0].tolist()
                ys = xy[:,1].tolist()
                gx.extend(xs)
                gy.extend(ys)
                if j < ng:
                    gx.append(nan)
                    gy.append(nan)
            lons.append(gx)
            lats.append(gy)

        else:     
            xy = np.array(list(shape.exterior.coords))
            xs = xy[:,0].tolist()
            ys = xy[:,1].tolist()
            lons.append(xs)
            lats.append(ys) 

    return lons,lats

In [6]:
IL_acs.columns

Index(['state', 'county', 'NAME_x', 'total_pop', 'pct_65_ove', 'pct_black',
       'pct_hispan', 'males', 'females', 'pct_bach', 'GEOID_x', 'STATEFP',
       'COUNTYFP', 'COUNTYNS', 'AFFGEOID', 'GEOID_y', 'NAME_y', 'LSAD',
       'ALAND', 'AWATER', 'GEOID_int', 'geometry'],
      dtype='object')

In [7]:
#Feed in the data for bokeh

lons, lats = gpd_bokeh(IL_acs)


source = ColumnDataSource(data=dict(
        x=lons, 
        y=lats,
        name = IL_acs['NAME_x'], #Add any columns you want to bokeh.
        population = IL_acs['total_pop'],
        Pct_Bach = IL_acs['pct_bach']))

In [8]:
#Create a color map
color_mapper = linear_cmap(field_name='Pct_Bach', #the field to map
                           palette=Spectral6, #the color to use
                           low=min(IL_acs['pct_bach']) , # The low and high bounds for your color map
                           high=max(IL_acs['pct_bach']))


In [9]:
#Add tools you want
TOOLS = "pan,wheel_zoom,reset,hover,save"

In [10]:
#Create a plot frame with size and title
map = figure(plot_width=450, plot_height=600,title="Education Attainment in Illinois", tools=TOOLS)

#Add the polygon patches
map.patches('x', 'y', source=source, line_color="white", line_width=0.1, color=color_mapper)

#Add the hover tool and the hover field to display
map.select_one(HoverTool).tooltips = [
    ('County Name', '@name'), #each tuple needs to follow this format.
    ('Population', '@population'), #the name after @ should be consistent with the variables names in ColumnDataSource
    ('% Bachelors degree', '@Pct_Bach')
]


#Add the colorbar to the map
color_bar = ColorBar(color_mapper=color_mapper['transform'], width=16, location=(0,0))
map.add_layout(color_bar, 'right')

In [11]:
#Show the map
show(map)

In [12]:
#You can export your map to a html file.

output_file("EDU_Map_IL.html")
show(map)

In [13]:
map.scatter(IL_acs.centroid.x, IL_acs.centroid.y)
show(map)